# Partie 2

## Initialisation du kernel

In [ ]:
# !pip install --upgrade oracledb

import getpass
from os import getenv
import pandas as pd
import oracledb
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

: 

In [5]:
# Nécessaire pour éviter les problèmes de session
class Connexion(object):
    def __init__(self, login, password):
        self.conn = oracledb.connect(
            user=login,
            password=password,
            host="oracle.iut-orsay.fr",
            port=1521,
            sid="etudom",
        )
        self.conn.autocommit = False

    def __enter__(self):
        self.conn.autocommit = False
        return self.conn

    def __exit__(self, *args):
        self.conn.close()

In [6]:
# La fonction ci-dessous est à utiliser pour exécuter une requête et stocker les résultats dans un dataframe Pandas sans afficher d’alerte.
# Vous pouvez vous en inspirer pour créer vos propres fonctions.
def requete_vers_dataframe(connexion_data, requete, valeurs = None):
    with Connexion(login=connexion_data['login'], password=connexion_data['password']) as connexion:
        warnings.simplefilter(action='ignore', category=UserWarning)
        if valeurs is not None:
            df = pd.read_sql(requete, connexion, params=valeurs)
        else:
            df = pd.read_sql(requete, connexion)
        warnings.simplefilter("always") 
        return df

In [7]:
# Saisir ci-dessous l'édition des JO qui vous a été attribuée. Cela correspond au LibelleHote dans la table HOTE de la base de données
# Par exemple EDITION_JO = "2020 Summer Olympics"
EDITION_JO = "1976 Summer Olympics"
# Saisir ci-dessous le login court de la base utilisée pour votre carnet
SCHEMA = "TPLUMAS"

In [8]:
# On demande à l'utilisateur son login et mot de passe pour pouvoir accéder à la base de données
if getenv("DB_LOGIN") is None:
    login = input("Login : ")
else:
    login = getenv("DB_LOGIN")
if getenv("DB_PASS") is None:
    password = getpass.getpass("Mot de passe : ")
else:
    password = getenv("DB_PASS")
conn = {'login': login, 'password': password}

In [9]:
# On vérifie que l'utilisateur est bien connecté à la base de données, que le schéma est bon, et qu'on trouve la bonne édition des JO
data = requete_vers_dataframe(conn, f"SELECT * FROM {SCHEMA}.HOTE WHERE LibelleHote LIKE (:libelle)",{"libelle":EDITION_JO})
id_hote = int(data.IDHOTE.iloc[0])
print(f"Identifiant de l’hôte : {id_hote}")

Identifiant de l’hôte : 19


## Présentation des jeux olympique

### Contextualisation de l'édition des JO

Les Jeux Olympique de 1976 se sont déroulés à Montréal au Canada. Ils se sont déroulés du 17 juillet 1976 au 1 août 1976.

Voici les différents lieux du site

<style>
    img{
        display: block;
        width: 60%;
        height: 60%;
        margin-left: auto;
        margin-right: auto;
    }
</style>

![img](https://img.olympics.com/images/image/private/t_s_16_9_g_auto/t_s_w1920/f_auto/primary/sfbgpwrndfqjeelbk0ug)


Ce sont les premiers jeux au Canada.
A noter que cette édition a vu 22 pays boycottés cette édition car la Nouvelle-Zélande avait envoyé sa délégation de rugby joué un match en Afrique du Sud, pays sous l'apartheid a cette époque.


### Information général de cette édition des JO

Il y a eu 6028 athlètes durant ces jo, avec 4781 hommes et 1247 femmes.
21 sports, 27 disciplines et 128 épreuves difféntes durant cette édition. 

Voici quelques statisques plus détaillés

In [10]:
requete = f"""
    SELECT 
        nomEvenement
    FROM 
        {SCHEMA}.EVENEMENT E
    INNER JOIN
        {SCHEMA}.HOTE H
    ON
        e.idHote = h.idHote
        
    WHERE
        libelleHote = '{EDITION_JO}'
"""

In [11]:
data = requete_vers_dataframe(conn,requete)
print("Voici la liste de toutes les épreuves")
data

Voici la liste de toutes les épreuves


,NOMEVENEMENT
0,"200 metres, Men"
1,"400 metres, Men"
2,"110 metres Hurdles, Men"
3,"400 metres Hurdles, Men"
4,"4 × 100 metres Relay, Men"
...,...
193,"Featherweight, Greco-Roman, Men"
194,"Heavyweight, Greco-Roman, Men"
195,"Lightweight, Men"
196,"Middle-Heavyweight, Men"


In [12]:
requete=f"""
SELECT DISTINCT
    nomDiscipline
FROM
    {SCHEMA}.DISCIPLINE D
INNER JOIN
    {SCHEMA}.EVENEMENT E
ON 
    d.codeDiscipline = e.codeDiscipline
INNER JOIN
    {SCHEMA}.HOTE H
ON
    e.idHote = h.idHote
WHERE
    libelleHote = '{EDITION_JO}'
"""

In [13]:
data = requete_vers_dataframe(conn,requete)
print("Voici la liste de toutes les disciplines")
data

Voici la liste de toutes les disciplines


,NOMDISCIPLINE
0,Athletics
1,Boxing
2,Cycling Track
3,Equestrian Dressage
4,Swimming
5,Wrestling
6,Basketball
7,Canoe Sprint
8,Modern Pentathlon
9,Volleyball


In [14]:
requete=f"""
SELECT DISTINCT
    nomSport
FROM
    {SCHEMA}.SPORT S
INNER JOIN
    {SCHEMA}.DISCIPLINE D
ON 
    s.codeSport = d.codeSport
INNER JOIN
    {SCHEMA}.EVENEMENT E
ON
    d.codeDiscipline = e.codeDiscipline
INNER JOIN
    {SCHEMA}.HOTE H
ON
    e.idHote = h.idHote
WHERE
    libelleHote = '{EDITION_JO}'
"""

In [15]:
data=requete_vers_dataframe(conn,requete)
print("Voici la liste des sports")
data

Voici la liste des sports


,NOMSPORT
0,Athletics
1,Boxing
2,Wrestling
3,Basketball
4,Modern Pentathlon
5,Volleyball
6,Cycling
7,Hockey
8,Judo
9,Shooting


In [16]:
requete = f"""

SELECT 
    COUNT(DISTINCT noc) AS NB
FROM
    (
        SELECT PI.noc,idEvent
        FROM {SCHEMA}.PARTICIPATION_INDIVIDUELLE PI
        UNION
        SELECT e.noc,idEvenement
        FROM {SCHEMA}.PARTICIPATION_EQUIPE PE
        INNER JOIN {SCHEMA}.EQUIPE E ON PE.idEquipe = E.idEquipe
    )
INNER JOIN
    {SCHEMA}.EVENEMENT E ON idEvent = E.idEvenement
INNER JOIN
    {SCHEMA}.HOTE H ON E.idHote = H.idHote
WHERE
    H.libelleHote = '{EDITION_JO}'
"""


In [17]:
nbPays=requete_vers_dataframe(conn,requete).NB.iloc[0]
print(f"nombre de pays : {nbPays}")

nombre de pays : 92


### Quelques statistiques concernant cette édition

In [1]:
'''
TODO : voici les statistique à faire

- proportion en pourcentage sur homme femme
- proportion des médailes remportés par NOC
- épreuve, discipline, sport ayant le plus d'athlète
- trouver les nouvelles épreuves
- que veux-dire répartition par genre ?
- répartition des différentes catégorie d'épreuve (genre tant de pourcent d'épreuve)

'''

"\nTODO : voici les statistique à faire\n\n- proportion en pourcentage sur homme femme\n- proportion des médailes remportés par NOC\n- épreuve, discipline, sport ayant le plus d'athlète\n- trouver les nouvelles épreuves\n- que veux-dire répartition par genre ?\n- répartition des différentes catégorie d'épreuve (genre tant de pourcent d'épreuve)\n\n"

In [ ]:
requete=f"""
SELECT COUNT(*)


"""

## Comparaison entre les éditions

## Performance de l'équipe de notre choix